In [ ]:
import cv2, os
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
for file in os.listdir('Images'):

    # Ouverture de l'image
    img = cv2.imread(f'Images/{file}', cv2.IMREAD_UNCHANGED) 
    src = img.copy()

    # On ne garde que la dimension 1 (= Vert)
    src[:,:,2] = np.zeros([src.shape[0], src.shape[1]])
    src[:,:,0] = np.zeros([src.shape[0], src.shape[1]])

    # Détéction de contours
    gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
    _,edge = cv2.threshold(gray, 50, 1, cv2.THRESH_BINARY)
    contours,_ = cv2.findContours(edge,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # On ne garde que le plus gros contour trouvé
    nb_cont = 0
    best = -1
    for i, contour in enumerate(contours):
        if len(contours[i]) > nb_cont:
            nb_cont = len(contours[i])
            best = i

    # On trouve les dimensions du contour
    xmin, xmax, ymin, ymax = 5000, 0, 5000, 0
    for i, contour in enumerate(contours[best]):
        if contour[0][0] > xmax:
            xmax = contour[0][0] 

        if contour[0][0] < xmin:
            xmin = contour[0][0] 

        if contour[0][1] > ymax:
            ymax = contour[0][1] 

        if contour[0][1] < ymin:
            ymin = contour[0][1]

    # Découpage de l'image avec les dimensions trouvées
    cropped = img[ymin:ymax, xmin:xmax]

    # Redimension de l'image par un multiple de 32
    x = cropped.shape[0]
    while x % 32 != 0:
        x -= 1
    resized = cv2.resize(cropped, (x, x), interpolation = cv2.INTER_AREA)

    # Sauvegarde de l'image
    cv2.imwrite(f'New_Images/{file}', resized)

    # # Pour découper l'image en 4
    # im1 = cropped[0 : cropped.shape[1]//2, 0 : cropped.shape[0]//2]
    # im2 = cropped[0 : cropped.shape[1]//2, cropped.shape[0]//2 + 1 : cropped.shape[0]]
    # im3 = cropped[cropped.shape[1]//2 + 1 : cropped.shape[1], 0 : cropped.shape[0]//2]
    # im4 = cropped[cropped.shape[1]//2 + 1 : cropped.shape[1], cropped.shape[0]//2 + 1 : cropped.shape[0]]

    # cropped_imgs = [im1, im2, im3, im4]
    # for i, image in enumerate(cropped_imgs):
    #     resized = cv2.resize(image, (640, 640), interpolation = cv2.INTER_AREA)
    #     cv2.imwrite(f'New_Images/{file[:-4]}_{i}.jpg', resized)